# Lab 12: Ransomware Simulation (Purple Team)

Build safe simulation tools for testing ransomware defenses.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab12_ransomware_simulation.ipynb)

**ETHICAL NOTE**: This lab is for authorized testing only. All simulations are safe and non-destructive.

## Learning Objectives
- Adversary emulation planning
- Safe simulation techniques
- Detection validation
- Gap analysis and reporting

In [ ]:
import tempfile
import shutil
from pathlib import Path
from dataclasses import dataclass, field
from typing import List, Dict
from datetime import datetime
from enum import Enum

## 1. Attack Scenario Generation

In [ ]:
class RansomwareFamily(Enum):
    LOCKBIT = "lockbit"
    BLACKCAT = "blackcat"
    CONTI = "conti"

@dataclass
class AttackScenario:
    """Ransomware attack scenario."""
    family: RansomwareFamily
    name: str
    description: str
    mitre_techniques: List[str]
    detection_opportunities: List[str]

# Define scenarios
SCENARIOS = {
    RansomwareFamily.LOCKBIT: AttackScenario(
        family=RansomwareFamily.LOCKBIT,
        name="LockBit 3.0 Simulation",
        description="Fast-encrypting RaaS with double extortion",
        mitre_techniques=["T1486", "T1490", "T1059.001", "T1567"],
        detection_opportunities=[
            "Shadow copy deletion commands",
            "Mass file extension changes",
            "Ransom note creation",
            "Large outbound transfers"
        ]
    ),
    RansomwareFamily.BLACKCAT: AttackScenario(
        family=RansomwareFamily.BLACKCAT,
        name="BlackCat/ALPHV Simulation",
        description="Rust-based cross-platform ransomware",
        mitre_techniques=["T1486", "T1490", "T1048", "T1070"],
        detection_opportunities=[
            "Unusual Rust binary execution",
            "Config file creation",
            "Service stop commands",
            "ESXi targeting behavior"
        ]
    )
}

print("Available Scenarios:")
for family, scenario in SCENARIOS.items():
    print(f"\n{scenario.name}")
    print(f"  Description: {scenario.description}")
    print(f"  Techniques: {scenario.mitre_techniques}")

## 2. Safe Simulator

In [ ]:
@dataclass
class SimulationConfig:
    """Safe simulation configuration."""
    target_directory: str
    file_extensions: List[str] = field(default_factory=lambda: [".txt", ".docx"])
    simulate_encryption: bool = True
    create_ransom_note: bool = True
    cleanup_after: bool = True

class SafeRansomwareSimulator:
    """
    Safe ransomware behavior simulator.
    
    SAFETY: Only operates in temp directories,
    no actual encryption, full audit logging.
    """
    
    def __init__(self, config: SimulationConfig):
        self.config = config
        self.audit_log = []
        self.created_files = []
        self._validate_safe_directory()
    
    def _validate_safe_directory(self):
        """Ensure we're in a safe directory."""
        path = Path(self.config.target_directory).resolve()
        temp_dir = Path(tempfile.gettempdir())
        
        if not str(path).startswith(str(temp_dir)):
            raise ValueError(f"Target must be in temp directory: {temp_dir}")
        
        path.mkdir(parents=True, exist_ok=True)
    
    def _log(self, action: str, details: Dict):
        self.audit_log.append({
            "timestamp": datetime.now().isoformat(),
            "action": action,
            **details
        })
    
    def setup_test_files(self, num_files: int = 5) -> List[str]:
        """Create test files for simulation."""
        files = []
        target = Path(self.config.target_directory)
        
        for i in range(num_files):
            for ext in self.config.file_extensions:
                filepath = target / f"test_file_{i}{ext}"
                filepath.write_text(f"Test content for {filepath.name}")
                files.append(str(filepath))
                self.created_files.append(str(filepath))
        
        self._log("SETUP", {"files_created": len(files)})
        return files
    
    def simulate_encryption(self, files: List[str]) -> Dict:
        """Simulate encryption (just renaming, NOT actual encryption)."""
        if not self.config.simulate_encryption:
            return {"simulated": False}
        
        affected = 0
        for filepath in files:
            original = Path(filepath)
            if original.exists():
                new_path = str(original) + ".encrypted"
                shutil.move(str(original), new_path)
                affected += 1
                self._log("ENCRYPT_SIM", {"file": str(original)})
        
        return {"simulated": True, "files_affected": affected}
    
    def create_ransom_note(self) -> str:
        """Create simulated ransom note."""
        if not self.config.create_ransom_note:
            return ""
        
        note = f"""
=== SIMULATION RANSOM NOTE ===
This is a SIMULATED ransom note for purple team testing.
No actual ransomware was deployed.

Test ID: PURPLE-TEAM-{datetime.now().strftime('%Y%m%d-%H%M%S')}
=== END SIMULATION ===
"""
        
        note_path = Path(self.config.target_directory) / "README_RESTORE_FILES.txt"
        note_path.write_text(note)
        self.created_files.append(str(note_path))
        self._log("RANSOM_NOTE", {"path": str(note_path)})
        
        return str(note_path)
    
    def cleanup(self):
        """Clean up all created files."""
        if not self.config.cleanup_after:
            return
        
        for filepath in self.created_files:
            path = Path(filepath)
            if path.exists():
                path.unlink()
            # Also check for .encrypted version
            enc_path = Path(filepath + ".encrypted")
            if enc_path.exists():
                enc_path.unlink()
        
        self._log("CLEANUP", {"files_removed": len(self.created_files)})

In [ ]:
# Run simulation
with tempfile.TemporaryDirectory() as tmpdir:
    config = SimulationConfig(
        target_directory=tmpdir,
        simulate_encryption=True,
        create_ransom_note=True,
        cleanup_after=True
    )
    
    simulator = SafeRansomwareSimulator(config)
    
    # Run simulation
    print("Running Safe Simulation:")
    print("=" * 40)
    
    files = simulator.setup_test_files(num_files=3)
    print(f"1. Created {len(files)} test files")
    
    result = simulator.simulate_encryption(files)
    print(f"2. Simulated encryption of {result['files_affected']} files")
    
    note = simulator.create_ransom_note()
    print(f"3. Created ransom note: {Path(note).name}")
    
    # Show audit log
    print(f"\nAudit Log: {len(simulator.audit_log)} entries")
    
    # Cleanup
    simulator.cleanup()
    print("4. Cleanup complete")

## 3. Detection Validation

In [ ]:
class DetectionStatus(Enum):
    DETECTED = "detected"
    MISSED = "missed"
    PENDING = "pending"

@dataclass
class DetectionTest:
    name: str
    technique_id: str
    description: str
    status: DetectionStatus = DetectionStatus.PENDING

class DetectionValidator:
    """Validate detection coverage."""
    
    def __init__(self):
        self.tests = [
            DetectionTest("Shadow Copy Deletion", "T1490", "vssadmin delete shadows"),
            DetectionTest("Mass File Modification", "T1486", "Rapid file extension changes"),
            DetectionTest("Ransom Note Creation", "T1486", "README/DECRYPT file creation"),
            DetectionTest("Service Termination", "T1489", "Backup service stops")
        ]
    
    def generate_gap_analysis(self) -> Dict:
        detected = sum(1 for t in self.tests if t.status == DetectionStatus.DETECTED)
        missed = sum(1 for t in self.tests if t.status == DetectionStatus.MISSED)
        
        return {
            "total_tests": len(self.tests),
            "detected": detected,
            "missed": missed,
            "coverage": f"{(detected/len(self.tests))*100:.0f}%" if self.tests else "0%",
            "gaps": [t.name for t in self.tests if t.status == DetectionStatus.MISSED]
        }

# Example validation
validator = DetectionValidator()

# Simulate test results
validator.tests[0].status = DetectionStatus.DETECTED
validator.tests[1].status = DetectionStatus.DETECTED
validator.tests[2].status = DetectionStatus.MISSED
validator.tests[3].status = DetectionStatus.MISSED

analysis = validator.generate_gap_analysis()

print("Detection Gap Analysis:")
print(f"  Coverage: {analysis['coverage']}")
print(f"  Detected: {analysis['detected']}/{analysis['total_tests']}")
print(f"  Gaps: {analysis['gaps']}")

## 4. Exercise Report

In [ ]:
def generate_exercise_report(scenario: AttackScenario, gap_analysis: Dict) -> str:
    """Generate purple team exercise report."""
    return f"""
# Purple Team Exercise Report

## Scenario
- **Name**: {scenario.name}
- **Family**: {scenario.family.value}
- **Description**: {scenario.description}

## MITRE ATT&CK Techniques Tested
{chr(10).join(f'- {t}' for t in scenario.mitre_techniques)}

## Detection Coverage
- **Overall Coverage**: {gap_analysis['coverage']}
- **Tests Detected**: {gap_analysis['detected']}
- **Tests Missed**: {gap_analysis['missed']}

## Detection Gaps
{chr(10).join(f'- {g}' for g in gap_analysis['gaps']) or 'None - full coverage!'}

## Recommendations
1. Implement missing detections for identified gaps
2. Tune existing rules to reduce false negatives
3. Re-run exercise after remediation
4. Document detection playbooks

---
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

# Generate report
scenario = SCENARIOS[RansomwareFamily.LOCKBIT]
report = generate_exercise_report(scenario, analysis)
print(report)

## Summary

We built a purple team ransomware simulation framework:

1. **Scenario Generation** - Model ransomware families
2. **Safe Simulation** - Non-destructive behavior emulation
3. **Detection Validation** - Test security controls
4. **Gap Analysis** - Identify missing coverage

### Safety Features:
- Only operates in temp directories
- No actual encryption (just renaming)
- Full audit logging
- Automatic cleanup

### Next Steps:
1. Integrate with detection systems for automated validation
2. Add more ransomware family profiles
3. Create scheduled exercise automation